# 2.1 Debugging

In [ ]:
import os
os.chdir('../home')

## Sections

- Introduction to debugging: importance and basic concepts.
- Configuration of the debugging environment in Visual Studio Code.
- Use of breakpoints to halt execution.
- Types of breakpoints: conditional and exception breakpoints.
- Inspection of variables and expressions during execution.
- Utilization of the debugging control panel.
- Step-by-step execution: step into, step over, and step out.
- Visualization tools: watch, variables, and call stack.
- Logging usage for debugging.
- Remote debugging and debugging in virtual environments.

## References
https://code.visualstudio.com/docs/python/python-quick-start
https://code.visualstudio.com/docs/python/debugging
https://code.visualstudio.com/docs/editor/debugging